### Criar nosso primeiro CATALOG

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS demo_catalog
COMMENT 'Catálogo de demonstração criado para o workshop de Unity Catalog';

### Criar um Schema (database) no catálogo

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS demo_catalog.bronze
COMMENT 'Schema de bronze para dados brutos e heterogêneos';

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS demo_catalog.silver
COMMENT 'Schema de silver para dados tratados';

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS demo_catalog.gold
COMMENT 'Schema de gold para dados que serão utilizados pelo negócio';